In [41]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import os
import glob
import scipy.ndimage as sim


import utils

Fig2

In [96]:
save_dir="./data/to_maya/fig2"
os.makedirs(save_dir, exist_ok=True)

In [97]:
def sigmoid(x):
    return 1/(1+np.exp(-x))
thres=0.1

In [140]:
fols=glob.glob("./data/images_1/2x2x2_final/sc3ss1sb3/*/*/")
rows=[]
for fol in fols:
    pfolname=os.path.basename(os.path.dirname(fol))
    seed=int(pfolname.split("seed=")[1].split("_")[0])

    ppfolname=os.path.basename(os.path.dirname(os.path.dirname(fol)))
    sep_col=float(ppfolname.split("sep_col=")[1].split("_")[0])
    sep_size=float(ppfolname.split("sep_size=")[1].split("_")[0])
    sep_bgcol=float(ppfolname.split("sep_bg_col=")[1].split("_")[0])
    if sep_bgcol not in [0.1,0.2,0.3]:
        continue

    pix_mse=np.load(os.path.join(fol,"mse_pix.npy"))
    d_000_001=pix_mse[0,1]
    d_000_010=pix_mse[0,2]
    d_001_011=pix_mse[1,3]
    d_010_011=pix_mse[2,3]

    d_000_100=pix_mse[0,4]
    d_001_101=pix_mse[1,5]
    d_010_110=pix_mse[2,6]
    d_011_111=pix_mse[3,7]

    d_100_101=pix_mse[4,5]
    d_100_110=pix_mse[4,6]
    d_101_111=pix_mse[5,7]
    d_110_111=pix_mse[6,7]


    config_file=glob.glob(os.path.join(fol,"*.yaml"))[0]
    config=utils.load_config(config_file)
    save_steps=np.array(config["save_steps"])
    plot_data=torch.load(os.path.join(fol,"plot_data.pth"))

    classprobs_tr=np.array(plot_data['classprobs_pred_tr'])[...,1]
    l_tr=np.array(plot_data['l_tr'])
    classprobs_te=np.array(plot_data['classprobs_pred_te'])[...,1]
    l_te=np.array(plot_data['l_te'])
    #print(classprobs_tr.shape,classprobs_te.shape)

    trajs=[]
    for l in range(8):
        if l in [0,1,2,4]:
            trajs.append(classprobs_tr[:,:,l_tr==l].mean(-1))
        else:
            trajs.append(classprobs_te[:,:,l_te==l].mean(-1))
    trajs=np.stack(trajs,axis=0)


    gts=np.array([[0.,0,0],[0,0,1],[0,1,0],[0,1,1],[1,0,0],[1,0,1],[1,1,0],[1,1,1]])
    trajs_smooth=sim.gaussian_filter1d(trajs,1.0,axis=1,truncate=6.0)
    accs=sigmoid(1/(np.abs(gts[:,None,:]-trajs_smooth)/thres+1e-9))

    trajs=trajs.transpose(0,2,1).reshape(-1)
    accs=accs.transpose(0,2,1).reshape(-1)

    rows.append([sep_col,sep_size,sep_bgcol,seed,
                 d_000_001,d_000_010,d_001_011,d_010_011,
                    d_000_100,d_001_101,d_010_110,d_011_111,
                    d_100_101,d_100_110,d_101_111,d_110_111,
                    *trajs,*accs,
                    ])
    #break
classnames=np.array(["000","001","010","011","100","101","110","111"])
dimnames=np.array(["color","size","bgcolor"])
header_traj=np.char.add("traj_",classnames)
header_traj=np.char.add(header_traj,"_")
header_traj=np.char.add(header_traj[:,None],dimnames[None,:]).flatten()
header_traj=np.char.add(header_traj,"_")
header_traj=np.char.add(header_traj[:,None],save_steps[None,:].astype(str)).flatten()
header_acc=np.char.add("acc_",classnames)
header_acc=np.char.add(header_acc,"_")
header_acc=np.char.add(header_acc[:,None],dimnames[None,:]).flatten()
header_acc=np.char.add(header_acc,"_")
header_acc=np.char.add(header_acc[:,None],save_steps[None,:].astype(str)).flatten()
header=["sep_col","sep_size","sep_bgcol","seed",
        "d_000_001","d_000_010","d_001_011","d_010_011",
        "d_000_100","d_001_101","d_010_110","d_011_111",
        "d_100_101","d_100_110","d_101_111","d_110_111",
        *header_traj,*header_acc
        ]
data=np.array(rows)

print(len(rows),len(rows[0]),len(header))

np.savetxt(os.path.join(save_dir,"fig2_data.csv"),data,header=",".join(header),delimiter=",",comments="",fmt='%.4f')

45 5776 5776


Fig3

In [73]:
save_dir="./data/to_maya/fig3"
os.makedirs(save_dir, exist_ok=True)

In [74]:
def get_i_learned(accs,consistent=3):
    assert consistent%2==1
    #find portion of acc where accs is positive for 3 values
    structure=np.ones(consistent)
    goods=sim.binary_erosion(accs,structure=structure)
    inds=np.nonzero(goods)[0]
    if len(inds)>=1:
        return np.min(inds)
    else:
        return None

In [75]:
fols=glob.glob("./data/images_1/2x2_final2/detailed/*/*/")
rows=[]
for fol in fols:
    pfolname=os.path.basename(os.path.dirname(fol))
    seed=int(pfolname.split("seed=")[1].split("_")[0])

    ppfolname=os.path.basename(os.path.dirname(os.path.dirname(fol)))
    sep_col=float(ppfolname.split("sep_col=")[1].split("_")[0])
    sep_size=float(ppfolname.split("sep_size=")[1].split("_")[0])

    pix_mse=np.load(os.path.join(fol,"mse_pix.npy"))
    d_00_01=pix_mse[0,1]
    d_00_10=pix_mse[0,2]
    d_01_11=pix_mse[1,3]
    d_10_11=pix_mse[2,3]

    config_file=glob.glob(os.path.join(fol,"*.yaml"))[0]
    config=utils.load_config(config_file)
    save_steps=np.array(config["save_steps"])
    plot_data=torch.load(os.path.join(fol,"plot_data.pth"))
    acc=plot_data["rights_te"].mean(1)
    coloracc=acc[:,0]
    sizeacc=acc[:,1]

    #calc time
    thres=0.9
    acc_smoothed=sim.gaussian_filter1d(acc,1.5,axis=0)
    i_learned_col=get_i_learned(acc_smoothed[:,0]>thres) 
    i_learned_size=get_i_learned(acc_smoothed[:,1]>thres)
    step_learned_col=-1 if i_learned_col is None else save_steps[i_learned_col]
    step_learned_size=-1 if i_learned_size is None else save_steps[i_learned_size]

    rows.append([sep_col,sep_size,seed,d_00_01,d_00_10,d_01_11,d_10_11,step_learned_col,step_learned_size,*coloracc,*sizeacc])
    #break
header_step_col=[str(s) for s in np.char.add("coloracc_",save_steps.astype(str))]
header_step_size=[str(s) for s in np.char.add("sizeacc_",save_steps.astype(str))]
header=["sep_col","sep_size","seed","d_00_01","d_00_10","d_01_11","d_10_11","step_learned_col","step_learned_size",*header_step_col,*header_step_size]
data=np.array(rows)
np.savetxt(os.path.join(save_dir,"fig3_data.csv"),data,header=",".join(header),delimiter=",",comments="",fmt='%.4f')

In [34]:
acc.shape

(50, 2)

In [20]:
len(fols)

245